In [ ]:
from tvDatafeed import TvDatafeed, Interval
import time

tv = TvDatafeed()  # ممكن تحط login هنا لو حبيت

gold_symbol = "XAUUSD"
usd_egp_symbol = "USDEGP"
exchange_gold = "OANDA"
exchange_egp = "FX_IDC"  # أو جرب "FOREXCOM"

while True:
    try:
        # بيانات الذهب
        gold_data = tv.get_hist(
            symbol=gold_symbol,
            exchange=exchange_gold,
            interval=Interval.in_1_minute,
            n_bars=1
        )

        # بيانات الدولار/جنيه
        egp_data = tv.get_hist(
            symbol=usd_egp_symbol,
            exchange=exchange_egp,
            interval=Interval.in_1_minute,
            n_bars=1
        )

        if gold_data is not None and not gold_data.empty and egp_data is not None and not egp_data.empty:
            gold_usd = gold_data['close'].iloc[-1]
            usd_egp = egp_data['close'].iloc[-1]
            gold_egp = gold_usd * usd_egp
            gold_gram_egp = gold_egp / 31.1

            print(f"🟡 Gold (USD): {gold_usd:.2f} | 💵 USD/EGP: {usd_egp:.2f} | 🇪🇬 Gold/Oz (EGP): {gold_egp:.2f} | ⚖️ 1g: {gold_gram_egp:.2f}")

        else:
            print("⚠️ No data received, retrying...")

    except Exception as e:
        print("❌ Error:", e)

    time.sleep(5)


In [ ]:

def get_gold_price():
    url = "https://gold-price-live.com"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    r = requests.get(url, headers=headers, timeout=10)
    soup = BeautifulSoup(r.text, 'html.parser')

    # حاول البحث بطرق متعددة حسب الصفحة
    row = soup.find("td", string=lambda text: text and "عيار 24" in text)
    if not row:
        raise ValueError("لم يتم العثور على السعر في الصفحة")

    price_elem = row.find_next_sibling("td")
    price_text = price_elem.text.strip().replace(",", "")
    
    try:
        price = float(price_text.split()[0])
    except ValueError:
        raise ValueError(f"القيمة غير قابلة للتحويل إلى رقم: {price_text}")
    
    return price

print("📈 بدء تتبع سعر الذهب 24K في مصر...\n")

while True:
    try:
        price = get_gold_price()
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"[{now}] 💰 Gold 24K: {price:.2f} EGP")
    except Exception as e:
        print("⚠️ Error:", e)
    time.sleep(300)
